In [1]:
import librosa
import os
import time
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Merge
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.layers.pooling import GlobalAveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import concatenate

Using TensorFlow backend.


In [ ]:
batch_size = 32
epochs = 100
exec_times = 5
augment_factor = 10

In [ ]:
# Load data from numpy file
songs =  np.load('data/features.npy')
genres =  np.load('data/label.npy')

# Split the dataset into training and test
X_train, X_test, y_train, y_test = train_test_split(
  songs, genres, test_size=0.1, stratify=genres)

# Split training set into training and validation
X_train, X_Val, y_train, y_val = train_test_split(
  X_train, y_train, test_size=1/6, stratify=y_train)

In [29]:
def splitsongs_melspect(X, y):
    temp_X = []
    temp_y = []

    for i, song in enumerate(X):
      song_slipted = np.split(song, augment_factor)
      for s in song_slipted:
        temp_X.append(s)
        temp_y.append(y[i])

    temp_X = np.array(temp_X)
    temp_y = np.array(temp_y)

    return temp_X, temp_y

In [30]:
# split the train, test and validation data in size 128x128
X_Val, y_val = splitsongs_melspect(X_Val, y_val)
X_test, y_test = splitsongs_melspect(X_test, y_test)
X_train, y_train = splitsongs_melspect(X_train, y_train)

In [31]:
kernel_size = 3
activation_func = Activation('relu')
input_shape = (128, 128)
inputs = Input(input_shape)

In [32]:
# Convolutional block_1
conv1 = Conv1D(32, kernel_size)(inputs)
act1 = activation_func(conv1)
bn1 = BatchNormalization()(act1)
pool1 = MaxPooling1D(pool_size=2, strides=2)(bn1)

In [33]:
# Convolutional block_2
conv2 = Conv1D(64, kernel_size)(pool1)
act2 = activation_func(conv2)
bn2 = BatchNormalization()(act2)
pool2 = MaxPooling1D(pool_size=2, strides=2)(bn2)

In [34]:
# Convolutional block_3
conv3 = Conv1D(128, kernel_size)(pool2)
act3 = activation_func(conv3)
bn3 = BatchNormalization()(act3)

In [35]:
# Global Layers
gmaxpl = GlobalMaxPooling1D()(bn3)
gmeanpl = GlobalAveragePooling1D()(bn3)
mergedlayer = concatenate([gmaxpl, gmeanpl], axis=1)

In [36]:
# Regular MLP
dense1 = Dense(512,
    kernel_initializer='glorot_normal',
    bias_initializer='glorot_normal')(mergedlayer)
actmlp = activation_func(dense1)
reg = Dropout(0.5)(actmlp)

In [37]:
dense2 = Dense(512,
    kernel_initializer='glorot_normal',
    bias_initializer='glorot_normal')(reg)
actmlp = activation_func(dense2)
reg = Dropout(0.5)(actmlp)

In [38]:
dense2 = Dense(10, activation='softmax')(reg)

In [39]:
cnn = Model(inputs=[inputs], outputs=[dense2])

In [40]:
print("\nTrain shape: {0}".format(X_train.shape))
print("Validation shape: {0}".format(X_Val.shape))
print("Test shape: {0}\n".format(X_test.shape))
print("Size of the CNN: %s\n" % cnn.count_params())


Train shape: (7500, 128, 128)
Validation shape: (1500, 128, 128)
Test shape: (1000, 128, 128)

Size of the CNN: 443498



In [41]:
cnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128)     0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 126, 32)      12320       input_2[0][0]                    
__________________________________________________________________________________________________
activation_2 (Activation)       multiple             0           conv1d_4[0][0]                   
                                                                 conv1d_5[0][0]                   
                                                                 conv1d_6[0][0]                   
                                                                 dense_4[0][0]                    
          

In [42]:
# Optimizers
sgd = keras.optimizers.SGD(lr=0.001, momentum=0.9, decay=1e-5, nesterov=True)
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-5)

In [43]:
# Compiler for the model
cnn.compile(loss=keras.losses.categorical_crossentropy,
  optimizer=sgd,
  metrics=['accuracy'])

In [44]:
# Early stop
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss',
  min_delta=0,
  patience=2,
  verbose=0,
  mode='auto')

In [45]:
# Fit the model
history = cnn.fit(X_train, y_train,
  batch_size=batch_size,
  epochs=epochs,
  verbose=1,
  validation_data=(X_Val, y_val))

Train on 7500 samples, validate on 1500 samples
Epoch 1/100
7500/7500 [==============================] - 16s 2ms/step - loss: 2.1237 - acc: 0.2389 - val_loss: 1.7405 - val_acc: 0.3733
Epoch 2/100
7500/7500 [==============================] - 15s 2ms/step - loss: 1.7722 - acc: 0.3539 - val_loss: 1.5586 - val_acc: 0.4500
Epoch 3/100
7500/7500 [==============================] - 15s 2ms/step - loss: 1.6355 - acc: 0.4120 - val_loss: 1.4790 - val_acc: 0.4713
Epoch 4/100
7500/7500 [==============================] - 15s 2ms/step - loss: 1.5344 - acc: 0.4452 - val_loss: 1.4270 - val_acc: 0.5007
Epoch 5/100
7500/7500 [==============================] - 15s 2ms/step - loss: 1.4411 - acc: 0.4813 - val_loss: 1.3647 - val_acc: 0.5200
Epoch 6/100
7500/7500 [==============================] - 15s 2ms/step - loss: 1.3659 - acc: 0.5169 - val_loss: 1.3078 - val_acc: 0.5333
Epoch 7/100
7500/7500 [==============================] - 16s 2ms/step - loss: 1.3153 - acc: 0.5392 - val_loss: 1.3173 - val_acc: 0.5433


In [46]:
score = cnn.evaluate(X_test, y_test, verbose=0)
score_val = cnn.evaluate(X_Val, y_val, verbose=0)

In [47]:
print("Score:", score[1])
print("Score Validation:", score_val[1])

Score: 0.645
Score Validation: 0.6160000002384186


In [48]:
cnn.save('model/cnn.h5')